### TFX Components

This notebook shows how to create pipeline that uses TFX components:

* CsvExampleGen
* StatisticsGen
* SchemaGen
* ExampleValidator
* Transform
* Trainer
* Evaluator

In [ ]:
import kfp

# Initializing the client
client = kfp.Client()

# ! Use kfp.Client(host='https://xxxxx.notebooks.googleusercontent.com/') if working from GCP notebooks (or local notebooks)

In [ ]:
input_data_uri = 'gs://<bucket>/tensorflow-tfx/tfx/components/testdata/external/csv'

#Only S3/GCS is supported. Replace with downloading component or GIT clone
#module_file = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/taxi_utils.py'
module_file = 'gs://<bucket>/tensorflow-tfx/tfx/examples/chicago_taxi_pipeline/taxi_utils.py'

In [ ]:
import json
from kfp.components import load_component_from_url

download_from_gcs_op = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/d013b8535666641ca5a5be6ce67e69e044bbf076/components/google-cloud/storage/download/component.yaml')

CsvExampleGen_op    = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/ExampleGen/CsvExampleGen/component.yaml')
StatisticsGen_op    = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/StatisticsGen/component.yaml')
SchemaGen_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/SchemaGen/component.yaml')
ExampleValidator_op = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/ExampleValidator/component.yaml')
Transform_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/Transform/component.yaml')
Trainer_op          = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/Trainer/component.yaml')
Evaluator_op        = load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/025c424a/components/tfx/Evaluator/component.yaml')

def tfx_pipeline(
    input_data_uri,
):
    download_task = download_from_gcs_op(
        input_data_uri,
    )

    examples_task = CsvExampleGen_op(
        input_base=download_task.output,
        input_config=json.dumps({
            "splits": [
                {'name': 'data', 'pattern': '*.csv'},
            ]
        }),
        output_config=json.dumps({
            "splitConfig": {
                "splits": [
                    {'name': 'train', 'hash_buckets': 2},
                    {'name': 'eval', 'hash_buckets': 1},
                ]
            }
        }),
    )
    
    statistics_task = StatisticsGen_op(
        examples_task.output,
    )
    
    schema_task = SchemaGen_op(
        statistics_task.output,
    )

    # Performs anomaly detection based on statistics and data schema.
    validator_task = ExampleValidator_op(
        stats=statistics_task.outputs['output'],
        schema=schema_task.outputs['output'],
    )

    # Performs transformations and feature engineering in training and serving.
    transform_task = Transform_op(
        input_data=examples_task.outputs['example_artifacts'],
        schema=schema_task.outputs['output'],
        module_file=module_file,
    )

    trainer_task = Trainer_op(
        module_file=module_file,
        examples=transform_task.outputs['transformed_examples'],
        schema=schema_task.outputs['output'],
        transform_output=transform_task.outputs['transform_output'],
        train_args=json.dumps({'num_steps': 10000}),
        eval_args=json.dumps({'num_steps': 5000}),
    )

    # Uses TFMA to compute a evaluation statistics over features of a model.
    model_analyzer = Evaluator_op(
        examples=examples_task.outputs['example_artifacts'],
        model_exports=trainer_task.outputs['output'],
        feature_slicing_spec=json.dumps({
            'specs': [
                {'column_for_slicing': ['trip_start_hour']},
            ],
        }),
    )


client.create_run_from_pipeline_func(
    tfx_pipeline,
    arguments=dict(
        input_data_uri=input_data_uri,
    ),
)